<a href="https://colab.research.google.com/github/mrrajatgarg/practice/blob/master/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

0  7			      c		       mp3.zip.001
1  8			      clip_file.csv    mp3.zip.002
2  9			      d		       mp3.zip.003
3  a			      e		       npy_files_TOTAL_train
4  annotations_final.csv      f		       sample_data
5  annotations_final_new.csv  features_id_mp3  wav_files_TOTAL
6  b			      mp3_all.zip


In [0]:
import pandas as pd
import numpy as np
import os
import shutil
import librosa
import time

In [0]:
directory_labels = '/content/'

In [0]:
# Load the annotations file (prior to execution check correctness of delimiter)
annotations = pd.read_csv(directory_labels + 'annotations_final.csv', sep="\t")

In [8]:
# check size of annotations
annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 190 entries, clip_id to mp3_path
dtypes: int64(189), object(1)
memory usage: 37.5+ MB


In [9]:
# ensure correct loading
annotations.head(5)

,clip_id,no voice,singer,duet,plucking,hard rock,world,bongos,harpsichord,female singing,...,rap,metal,hip hop,quick,water,baroque,women,fiddle,english,mp3_path
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
2,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
3,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...
4,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,f/american_bach_soloists-j_s__bach_solo_cantat...


In [0]:
# merge synonym tags
synonyms = [['beat', 'beats'],
            ['chant', 'chanting'],
            ['choir', 'choral'],
            ['classical', 'clasical', 'classic'],
            ['drum', 'drums'],
            ['electro', 'electronic', 'electronica', 'electric'],
            ['fast', 'fast beat', 'quick'],
            ['female', 'female singer', 'female singing', 'female vocals', 'female vocal', 'female voice', 'woman', 'woman singing', 'women'],
            ['flute', 'flutes'],
            ['guitar', 'guitars'],
            ['hard', 'hard rock'],
            ['harpsichord', 'harpsicord'],
            ['heavy', 'heavy metal', 'metal'],
            ['horn', 'horns'],
            ['india', 'indian'],
            ['jazz', 'jazzy'],
            ['male', 'male singer', 'male vocal', 'male vocals', 'male voice', 'man', 'man singing', 'men'],
            ['no beat', 'no drums'],
            ['no singer', 'no singing', 'no vocal','no vocals', 'no voice', 'no voices', 'instrumental'],
            ['opera', 'operatic'],
            ['orchestra', 'orchestral'],
            ['quiet', 'silence'],
            ['singer', 'singing'],
            ['space', 'spacey'],
            ['string', 'strings'],
            ['synth', 'synthesizer'],
            ['violin', 'violins'],
            ['vocal', 'vocals', 'voice', 'voices'],
            ['strange', 'weird']]

In [0]:
# replace all synonym words with the first in the list constructed above
for synonym_list in synonyms:
    annotations[synonym_list[0]] = annotations[synonym_list].max(axis=1)
    annotations.drop(synonym_list[1:], axis=1, inplace=True)

In [12]:
# check results
# previously: Columns: 190 entries, clip_id to mp3_path &  memory usage: 37.5+ MB
# now: Columns: 136 entries, clip_id to mp3_path & memory usage: 26.8+ MB
annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 136 entries, clip_id to mp3_path
dtypes: int64(135), object(1)
memory usage: 26.8+ MB


In [0]:
# keep clip_id and mp3_path in a different matrix to bind later
annotations_ids = annotations[['clip_id', 'mp3_path']]

In [0]:
# drop clip_id and mp3_path to proceed with distribution counts
annotations = annotations[annotations.columns.difference(['clip_id', 'mp3_path'])]

In [15]:
annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25863 entries, 0 to 25862
Columns: 134 entries, acoustic to world
dtypes: int64(134)
memory usage: 26.4 MB


In [0]:
# tags' distribution
tags_distro = annotations.sum(axis=0)
# sort tags according to their occurencies 
tags_distro.sort_values(axis=0, inplace=True, ascending = False)

In [0]:
#find 50 top tags
topindex, topvalues = list(tags_distro.index[:50]), tags_distro.values[:50]

In [0]:
# list of columns to remove from annotation file (not common labels)
rem_cols_index =list(tags_distro.index[50:])

In [19]:
#check how many columns we are about to remove
len(rem_cols_index) 

84

In [0]:
# keep only the 50 most common tags
annotations.drop(rem_cols_index, axis=1, inplace=True)

In [0]:
# bind back clip_id and mp3_path
annotations['clip_id'] = annotations_ids['clip_id']
annotations['mp3_path'] = annotations_ids['mp3_path']

In [22]:
# ensure correct binding
annotations.head(5)

,ambient,baroque,bass,beat,cello,chant,choir,classical,country,dance,...,solo,strange,string,synth,techno,trance,violin,vocal,clip_id,mp3_path
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,2,f/american_bach_soloists-j_s__bach_solo_cantat...
1,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,6,f/american_bach_soloists-j_s__bach_solo_cantat...
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,10,f/american_bach_soloists-j_s__bach_solo_cantat...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,f/american_bach_soloists-j_s__bach_solo_cantat...
4,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,12,f/american_bach_soloists-j_s__bach_solo_cantat...


In [0]:
# check results
# previously: Columns: 136 entries, clip_id to mp3_path & memory usage: 26.8+ MB
# now: Columns: 52 entries, clip_id to mp3_path &  memory usage: 10.1 MB
annotations.info()

In [24]:
!ls

0  7			      c		       mp3.zip.001
1  8			      clip_file.csv    mp3.zip.002
2  9			      d		       mp3.zip.003
3  a			      e		       npy_files_TOTAL_train
4  annotations_final.csv      f		       sample_data
5  annotations_final_new.csv  features_id_mp3  wav_files_TOTAL
6  b			      mp3_all.zip


In [0]:
annotations.to_csv(directory_labels + 'annotations_final_new2.csv', sep=',')

In [28]:
!ls

0  7			       b		mp3_all.zip
1  8			       c		mp3.zip.001
2  9			       clip_file.csv	mp3.zip.002
3  a			       d		mp3.zip.003
4  annotations_final.csv       e		npy_files_TOTAL_train
5  annotations_final_new2.csv  f		sample_data
6  annotations_final_new.csv   features_id_mp3	wav_files_TOTAL


In [0]:
new_annotations = pd.read_csv(directory_labels + 'annotations_final_new2.csv', sep="\t")

In [33]:
annotations.head(5)

,ambient,baroque,bass,beat,cello,chant,choir,classical,country,dance,...,solo,strange,string,synth,techno,trance,violin,vocal,clip_id,mp3_path
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,2,f/american_bach_soloists-j_s__bach_solo_cantat...
1,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,6,f/american_bach_soloists-j_s__bach_solo_cantat...
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,10,f/american_bach_soloists-j_s__bach_solo_cantat...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,f/american_bach_soloists-j_s__bach_solo_cantat...
4,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,12,f/american_bach_soloists-j_s__bach_solo_cantat...


# testing goes here

In [0]:
annotations_newest = annotations.head(5)

In [37]:
annotations_newest

,ambient,baroque,bass,beat,cello,chant,choir,classical,country,dance,...,solo,strange,string,synth,techno,trance,violin,vocal,clip_id,mp3_path
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,2,f/american_bach_soloists-j_s__bach_solo_cantat...
1,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,6,f/american_bach_soloists-j_s__bach_solo_cantat...
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,10,f/american_bach_soloists-j_s__bach_solo_cantat...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,f/american_bach_soloists-j_s__bach_solo_cantat...
4,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,12,f/american_bach_soloists-j_s__bach_solo_cantat...


In [0]:
clip_address = annotations_newest[['mp3_path']]

In [44]:
clip_address

,mp3_path
0,f/american_bach_soloists-j_s__bach_solo_cantat...
1,f/american_bach_soloists-j_s__bach_solo_cantat...
2,f/american_bach_soloists-j_s__bach_solo_cantat...
3,f/american_bach_soloists-j_s__bach_solo_cantat...
4,f/american_bach_soloists-j_s__bach_solo_cantat...


In [0]:
# drop clip_id and mp3_path to proceed with distribution counts
annotations_newest = annotations_newest[annotations_newest.columns.difference(['clip_id', 'mp3_path'])]

In [46]:
annotations_newest

,ambient,baroque,bass,beat,cello,chant,choir,classical,country,dance,...,slow,soft,solo,strange,string,synth,techno,trance,violin,vocal
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


In [0]:
numpy_array = annotations_newest.values

In [51]:
numpy_array

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0]])

In [52]:
print(numpy_array.shape)

(5, 50)


In [61]:
for index, row in clip_address.iterrows() :
  var = row['mp3_path']
  print(var)
  print(index)
  print(numpy_array[index].shape)

f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-30-59.mp3
0
(50,)
f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-146-175.mp3
1
(50,)
f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-262-291.mp3
2
(50,)
f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-291-320.mp3
3
(50,)
f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-320-349.mp3
4
(50,)


In [0]:
list_data = []
savename = "file.npy"

for index, row in clip_address.iterrows() :
  path = '/content/' + row['mp3_path']
  wavedata, samplerate = librosa.load(path, sr=2108)
  wavedata = wavedata[np.newaxis, :]
  list_data.append(wavedata)
  
np.save("list_data_demo" + savename, list_data)

In [0]:
list_data = np.load("list_data_demo" + savename)

In [66]:
list_data.shape

(5, 1, 61394)

In [0]:
list_data = [] 
classes = [] 
load = None
savename = "offseti2-duration2.npy"

start = time.time()

for c in classes_temp:
  for root, subdirs, files in os.walk(os.path.join(AUDIO_PATH, c)):
    for file in files:
      for i in range(0, 20, 2):
        filepath = os.path.join(root, file)    
        wavedata, samplerate = librosa.load(filepath, sr=None, mono=True, offset=i, duration=2)    
        wavedata = wavedata[np.newaxis, :]
        list_data.append(wavedata)
        classes.append(c)

np.save("list_data-" + savename, list_data)
np.save("classes-" + savename, classes)
print(time.time() - start)

In [68]:
!ls

0  8			       clip_file.csv	       mp3.zip.002
1  9			       d		       mp3.zip.003
2  a			       e		       npy_files_TOTAL_train
3  annotations_final.csv       f		       sample_data
4  annotations_final_new2.csv  features_id_mp3	       wav_files_TOTAL
5  annotations_final_new.csv   list_data_demofile.npy
6  b			       mp3_all.zip
7  c			       mp3.zip.001


In [0]:
from google.colab import files

In [0]:
files.download('list_data_demofile.npy')